In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

from tensorflow.keras.layers import Dense, LSTM, Input, TimeDistributed
from tensorflow.keras.models import Model

In [2]:
train_x = []
train_y = []

for i in range(1,3500):
    tmp = pd.read_csv('./single_label_dataset/beat/' + str(i) + '.csv.gzip', compression='gzip').to_numpy()
    train_x.append(tmp[:31])
    train_y.append(tmp[-1])

train_x = np.array(train_x)
train_y = np.array(train_y)



In [3]:
train_x.shape

(3499, 31, 23)

In [75]:
model = tf.keras.models.Sequential()
model.add(LSTM(units=23, activation='softmax', input_shape=(31,23), dropout=0.3, recurrent_dropout=0.3))

In [76]:
model.compile(loss = tf.keras.losses.CategoricalCrossentropy(), optimizer=tf.keras.optimizers.Adagrad(), metrics=['accuracy'])
callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5)

In [77]:
with tf.device("/gpu:0"):
    model.fit(train_x,train_y, epochs=4000, callbacks=callback, validation_split=0.1)

Epoch 1/4000
99/99 [==============================] - 21s 201ms/step - loss: 3.1244 - accuracy: 0.0959 - val_loss: 3.1212 - val_accuracy: 0.1343
Epoch 2/4000
99/99 [==============================] - 19s 189ms/step - loss: 3.1187 - accuracy: 0.1321 - val_loss: 3.1165 - val_accuracy: 0.1543
Epoch 3/4000
99/99 [==============================] - 19s 196ms/step - loss: 3.1141 - accuracy: 0.1426 - val_loss: 3.1125 - val_accuracy: 0.1600
Epoch 4/4000
99/99 [==============================] - 19s 191ms/step - loss: 3.1089 - accuracy: 0.1616 - val_loss: 3.1089 - val_accuracy: 0.1600
Epoch 5/4000
99/99 [==============================] - 19s 192ms/step - loss: 3.1067 - accuracy: 0.1708 - val_loss: 3.1055 - val_accuracy: 0.1686
Epoch 6/4000
99/99 [==============================] - 19s 194ms/step - loss: 3.1037 - accuracy: 0.1588 - val_loss: 3.1024 - val_accuracy: 0.1714
Epoch 7/4000
99/99 [==============================] - 19s 193ms/step - loss: 3.1009 - accuracy: 0.1543 - val_loss: 3.0994 - val_ac

In [ ]:
model.save('grad_simple.h5')

In [21]:
import functools
top3_acc = functools.partial(tf.keras.metrics.top_k_categorical_accuracy, k=3)

dilla.wav


In [23]:
truth_loop = [7, 22, 5, 22, 13, 22, 5, 5, 5, 22, 7, 22, 13, 7, 5, 22]

prd = [0, 22, 22, 22, 16, 22, 22, 22, 5,  22, 0 , 22, 1 , 0,  22, 22,\
       0, 22, 22, 22, 1,  22, 22, 22, 22, 22, 0 , 22, 1 , 0,  22, 22, \
       0, 22, 22, 22, 1,  22, 22, 22, 5,  22, 0 , 22, 1 , 0,  22, 22, \
       0, 22, 22, 22, 1,  22, 22, 22, 5,  22, 10, 22, 1 , 0,  22, 22, \
       0, 22, 22, 22, 13, 22, 0,  22, 22, 22, 0,  22, 1 , 0,  22, 22, \
       0, 22, 22, 22, 16, 22, 22, 22, 22, 22, 0,  22, 1 , 0,  22, 22, \
       0, 22, 22, 22, 1,  22, 22, 22, 5,  22, 0,  22, 1 , 10, 22, 22, \
       0, 22, 22, 22, 1,  22, 22, 22, 22, 22, 0,  22, 1 , 0,  22, 22, \
       0, 22, 22, 22, 1,  22, 0,  22, 7,  22, 0,  22, 16, 0,  22, 22, \
       0, 22, 22, 22, 1,  22, 22, 22, 22, 22, 0,  22, 1,  0,  22, 22, \
       0, 22, 22, 22, 1,  22, 22, 22, 22, 22, 0,  22, 1,  0,  22, 22, \
       0, 22, 22, 22, 13, 22, 22, 22, 5,  22, 0,  22, 1,  0,  22, 22]

ground_truth = [] # loop * 12


In [34]:
prd_input = np.zeros((prd.__len__(), 23))

i = 0
for p in prd:
    prd_input[i][int(p)] = 1
    i += 1


In [37]:
prd_input.shape

(192, 23)

In [82]:
change = []
for i in range(160):
    change.append(model.predict(np.expand_dims(prd_input[i:i+31], axis=0)))

In [83]:
change

[array([[0.02316115, 0.02308333, 0.01801805, 0.02211945, 0.02165429,
         0.02640162, 0.01891942, 0.02454149, 0.02005574, 0.02078076,
         0.02028883, 0.02222886, 0.02273456, 0.0237899 , 0.01897601,
         0.02068872, 0.02234565, 0.01872237, 0.01885828, 0.02284385,
         0.02049321, 0.01883128, 0.02277361]], dtype=float32),
 array([[0.02311678, 0.02301921, 0.01804894, 0.02212713, 0.02162825,
         0.02637068, 0.01896616, 0.02461283, 0.02006377, 0.02070423,
         0.02030526, 0.02212684, 0.02281519, 0.02372375, 0.01895065,
         0.02072694, 0.02231691, 0.01879947, 0.01882615, 0.02283505,
         0.02053733, 0.01883787, 0.02281114]], dtype=float32),
 array([[0.02047913, 0.02341809, 0.02207357, 0.01956206, 0.02301363,
         0.02461947, 0.0230291 , 0.02389567, 0.0218257 , 0.01950832,
         0.02087427, 0.01824366, 0.02220892, 0.02500943, 0.02256666,
         0.02042628, 0.02030284, 0.01937939, 0.02239788, 0.02241989,
         0.01865255, 0.02044089, 0.02609533]],

In [51]:
cnt = 0
asdf = 0
for i in train_y:
    cnt += 1
    if np.argmax(i) == 22:
        asdf += 1

In [53]:
asdf

903